<a href="https://colab.research.google.com/github/Ishaan6233/Emotion-Detection-NeuralNetwork/blob/main/Emotion_Detection_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Convolution Neural Networks (CNNs) are a type of deep learning neural network architecture specifically designed for processing grid like data, as such as an image and videos. CNNs have revolutionized the field of computer vision and are widely used for various tasks, including image classification, object detection, facial recognition, and image generation.

In [1]:
!pip install keras
!pip install tensorflow
!pip install tqdm
!pip install keras_preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 795.9 kB/s eta 0:00:00


In [23]:
import os
import tarfile
import pandas as pd
import numpy as np
from keras.utils import to_categorical  # converts a class vector to a binary-class matrix (one-hot-encoding)
from keras_preprocessing.image import load_img # Loads an image from a file and returns it as a PIL image
from keras.models import Sequential # Creates a linear stack of layers for the neural network
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import matplotlib.pyplot as plt
from keras.models import model_from_json
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Data Preperation

In [4]:
def extract_tar_gz(tar_gz_path, extract_path):
    if not os.path.exists(extract_path):
        os.makedirs(extract_path, exist_ok=True)
    with tarfile.open(tar_gz_path, 'r:gz') as tar_ref:
        tar_ref.extractall(extract_path)
    print(f"Extracted {tar_gz_path} to {extract_path}")

In [5]:
tar_gz_file_path = '/content/drive/MyDrive/Colab Notebooks/images.tar.gz'
extract_dir = '/content/drive/MyDrive/Colab Notebooks/images'

In [6]:
train_dir = os.path.join(extract_dir, "images", "train")
validation_dir = os.path.join(extract_dir, "images", "validation")

if os.listdir(train_dir):
    print("Train directory is not empty. Skipping extraction.")
elif os.listdir(validation_dir):
    print("Validation directory is not empty. Skipping extraction.")
else:
    extract_tar_gz(tar_gz_file_path, extract_dir)

Train directory is not empty. Skipping extraction.


In [7]:
TRAIN_DIR = os.path.join(extract_dir, 'images/train')
TEST_DIR = os.path.join(extract_dir, 'images/validation')

In [8]:
def createdataframe(dir):
    if not os.path.exists(dir):
        raise FileNotFoundError(f"The directory {dir} does not exist.")
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        label_path = os.path.join(dir, label)
        if os.path.isdir(label_path):
            for imagename in os.listdir(label_path):
                image_paths.append(os.path.join(label_path, imagename))
                labels.append(label)
            print(label, "completed")
    return image_paths, labels

Create test and train dataframes

In [9]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)
train.head()

surprise completed
happy completed
disgust completed
angry completed
fear completed
neutral completed
sad completed


,image,label
0,/content/drive/MyDrive/Colab Notebooks/images/...,surprise
1,/content/drive/MyDrive/Colab Notebooks/images/...,surprise
2,/content/drive/MyDrive/Colab Notebooks/images/...,surprise
3,/content/drive/MyDrive/Colab Notebooks/images/...,surprise
4,/content/drive/MyDrive/Colab Notebooks/images/...,surprise


In [10]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)
test.head()

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


,image,label
0,/content/drive/MyDrive/Colab Notebooks/images/...,angry
1,/content/drive/MyDrive/Colab Notebooks/images/...,angry
2,/content/drive/MyDrive/Colab Notebooks/images/...,angry
3,/content/drive/MyDrive/Colab Notebooks/images/...,angry
4,/content/drive/MyDrive/Colab Notebooks/images/...,angry


Image Processing

In [22]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
target_size = (48, 48)

def process_image(image):
    img = load_img(image, color_mode='grayscale', target_size=target_size)
    img = np.array(img)
    return img

def extract_features(image_paths, target_size=(48, 48)):
    features = []
    def process_image(image_path):
        img = load_img(image_path, color_mode='grayscale', target_size=target_size)
        img_array = img_to_array(img)
        img_array = img_array / 255.0  # Normalize the image
        return img_array

    with ThreadPoolExecutor() as executor:
        future_to_image = {executor.submit(process_image, image): image for image in image_paths}
        for future in tqdm(as_completed(future_to_image), total=len(image_paths), desc="Processing Images"):
            img = future.result()
            features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), target_size[0], target_size[1], 1)
    return features

In [12]:
train_features = extract_features(train['image'])

Processing Images: 100%|██████████| 28821/28821 [21:59<00:00, 21.84it/s]


In [13]:
test_features = extract_features(test['image'])

Processing Images: 100%|██████████| 7066/7066 [05:22<00:00, 21.90it/s]


In [14]:
x_train = train_features/255.0
x_test = test_features/255.0

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [16]:
y_train = le.transform(train['label'])/255.0
y_test = le.transform(test['label'])/255.0

In [17]:
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [18]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()
# Convolution Layers
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
# Fully Connected Layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(7, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 46, 46, 128)         │           1,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 23, 23, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 23, 23, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 21, 21, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 10, 10, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 10, 10, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 8, 8, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 4, 4, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 4, 4, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       4,194,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 7)                   │           1,799 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,804,551 (22.14 MB)

 Trainable params: 5,804,551 (22.14 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.fit(x= x_train, y= y_train, batch_size=128, epochs=1, validation_data=(x_test, y_test))

226/226 ━━━━━━━━━━━━━━━━━━━━ 1119s 5s/step - accuracy: 1.0000 - loss: 0.0324 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [35]:
model_json = model.to_json()
with open("emotiondetector.json", "w") as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [36]:
with open("emotiondetector.json", "r") as json_file:
  model_json = json_file.read()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [37]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [38]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print("\nTest accuracy:", test_acc)

221/221 - 66s - 298ms/step - accuracy: 1.0000 - loss: 0.0000e+00

Test accuracy: 1.0


In [39]:
def preprocess_image(image_path, target_size=(48, 48)):
    img = load_img(image_path, color_mode='grayscale')
    img = img.resize(target_size, Image.ANTIALIAS)  # Resize image to target size
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize the image
    return img_array

# Predict function
def predict(image_path, model, label, target_size=(48, 48)):
    img = preprocess_image(image_path, target_size)
    pred = model.predict(img)
    pred_label = label[np.argmax(pred)]
    return pred_label

In [46]:
import cv2
# Webcam emotion detection
def webcam_emotion_detection(model, label):
    haar_file = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    face_cascade = cv2.CascadeClassifier(haar_file)

    webcam = cv2.VideoCapture(0)
    while True:
        ret, frame = webcam.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]
            face = cv2.resize(face, (48, 48))
            img = extract_features([face])  # Wrap the face in a list to pass to extract_features
            pred = model.predict(img)
            prediction_label = label[pred.argmax()]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, '%s' % (prediction_label), (x, y-10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255))
        cv2.imshow("Output", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    webcam.release()
    cv2.destroyAllWindows()

In [42]:
path = '/content/drive/MyDrive/Colab Notebooks/images/images/train/angry/'
image_path = path+'27.jpg'
print("Original image is of angry")
prediction = predict(image_path, model, label)
print("Predicted image is of ", prediction)

Original image is of angry


<ipython-input-39-106e427ee66a>:3: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(target_size, Image.ANTIALIAS)  # Resize image to target size


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
Predicted image is of  angry


In [43]:
image_path = '/content/drive/MyDrive/Colab Notebooks/images/images/trials/maddison.jpg'
print("Original image is of ---")
prediction = predict(image_path, model, label)
print("Predicted image is of ", prediction)

Original image is of ---


<ipython-input-39-106e427ee66a>:3: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(target_size, Image.ANTIALIAS)  # Resize image to target size


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Predicted image is of  angry


In [47]:
webcam_emotion_detection(model, label)

here
